# JSON Mangler

**A script** to convert the file structure of a firebase database into another structure.

### Usage 
- Load Jupyter Notebook
- Import all libraries
- Run cell (5) with paramaters x=100, y=250

In [1]:
# json-mangler: Convert firebase database into txt file with stations and timestamps

import json
import os

# FLAW: ODENPLAN MULTIPLE START STOP IN SAME DAY
# BIG FLAW: MULTIPLE START STOP IN SAME DAY!! SAVE DIFFERENTLY

"""

Firebase data structure:
Plattform
 --> Odenplan
     --> Date Time
         --> Action
    Fridhemsplan
    --> Date Time
         --> Action
     
Goal data structure
structure = {
    'date': [(station, start), ...]
    }
}

Convert into txt file:
2021-10-20 stations-AM.txt

1. Gullmarsplan: xx:xx:xx - xx:xx:xx
2. Skanstull: xx:xx:xx - xx:xx:xx

"""

"\n\nFirebase data structure:\nPlattform\n --> Odenplan\n     --> Date Time\n         --> Action\n    Fridhemsplan\n    --> Date Time\n         --> Action\n     \nGoal data structure\nstructure = {\n    'date': [(station, start), ...]\n    }\n}\n\nConvert into txt file:\n2021-10-20 stations-AM.txt\n\n1. Gullmarsplan: xx:xx:xx - xx:xx:xx\n2. Skanstull: xx:xx:xx - xx:xx:xx\n\n"

In [2]:
datafile = './input/Firebase/firebase.json'

with open(datafile, 'r') as f:
    data = json.load(f)

# Convert into datetime structure with station + start and stop
structure = {}

for station in data['stations'].keys():
    
    # Get timestamps
    for datetime in data['stations'][station].keys():
        
        # Split datetime into date and time
        s_date, s_time = datetime.split(' ')
        
        # Label action as start or end
        action = data['stations'][station][datetime]['action']
        action = 'start' if action == 'On plattform' else 'stop'
        
        # Do not consider stop-actions
        if action == 'stop': continue
        
        # Add structure skeleton
        if s_date not in structure:
            structure[s_date] = []
        
        # Add data
        s_tuple = (station, s_time)
        structure[s_date].append(s_tuple)

In [4]:
print(structure['2021-10-12'])

[('Alvik', '07:50:30'), ('Fridhemsplan', '08:04:12'), ('Gamla Stan', '08:05:31'), ('Gullmarsplan', '07:51:28'), ('Kristineberg', '07:56:01'), ('Medborgarplatsen', '07:58:23'), ('Odenplan', '08:12:36'), ('Skanstull', '07:55:06'), ('Slussen', '08:01:46'), ('St Eriksplan', '08:08:06'), ('Thorildsplan', '07:59:29')]


In [6]:
route1 = ['Gullmarsplan', 'Skanstull', 'Medborgarplatsen', 'Slussen', 'Gamla Stan', 'T-Centralen', 'Hötorget', 'Rådmansgatan']

for t in structure['2021-10-12']:
    if t[0] in route1:
        print(t)
    
    

('Gamla Stan', '08:05:31')
('Gullmarsplan', '07:51:28')
('Medborgarplatsen', '07:58:23')
('Skanstull', '07:55:06')
('Slussen', '08:01:46')


In [20]:
route1 = ['Gullmarsplan', 'Skanstull', 'Medborgarplatsen', 'Slussen', 'Gamla Stan', 'T-Centralen', 'Hötorget', 'Rådmansgatan']

# For every date, construct a "station report"
for s_date in structure.keys():
    
    # Extract stations for routes
    route1_tuples = []
    route2_tuples = []
    
    for s_tuple in structure[s_date]:
        if s_tuple[0] in route1:
            route1_tuples.append(s_tuple)
        else:
            route2_tuples.append(s_tuple)
            
    # Sort by start time
    route1_tuples.sort(key=lambda x:x[1])
    route2_tuples.sort(key=lambda x:x[1])
    
    # CREATE STATION REPORT
    """
    ------------[ STATION REPORT ]------------
    Date: 2021-10-10
    
    Route "Alice":
    - Gullmarsplan: 07:12:34
    - Skanstull: 07:15:35
    
    Route "Alicia":
    - Islandstorget: 07:03:21
    - Ängbyplan: 07:06:39
    """
    
    # Create header
    header = '------------[ STATION REPORT ]------------\n'
    header += f'Date: {s_date}\n\n'
    
    contents = header
    
    # Add stations to file contents
    if len(route1_tuples) > 0:
        contents += 'Route "Alice"\n'
    
        for t in route1_tuples:
            contents += f'-{t[0]}: {t[1]}\n'
            
        contents += '\n'
            
    if len(route2_tuples) > 0:
        contents += 'Route "Alicia"\n'
    
        for t in route2_tuples:
            contents += f'-{t[0]}: {t[1]}\n' 
    
    # Enter report in correct folder
    folderpath = f'./by-date/{s_date}'
    
    # Create folder if not exists
    if not os.path.exists(folderpath):
        os.makedirs(folderpath)
    
    filepath = folderpath + '/station_report.txt'
  
    with open(filepath, 'w') as f:
        f.write(contents)
        print(f'Stations saved to {folderpath}.')
    

Stations saved to ./by-date/2021-09-29.
Stations saved to ./by-date/2021-09-30.
Stations saved to ./by-date/2021-10-04.
Stations saved to ./by-date/2021-10-05.
Stations saved to ./by-date/2021-10-06.
Stations saved to ./by-date/2021-10-07.
Stations saved to ./by-date/2021-10-11.
Stations saved to ./by-date/2021-09-28.
Stations saved to ./by-date/2021-10-01.
Stations saved to ./by-date/2021-10-08.
Stations saved to ./by-date/2021-10-12.
Stations saved to ./by-date/2021-09-27.
Stations saved to ./by-date/2021-09-24.
Stations saved to ./by-date/2021-09-23.
